In [5]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
from titiler.xarray.factory import xarray_open_dataset, update_dataset
import morecantile
from rio_tiler.io import XarrayReader

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

In [6]:
search_dict: Dict[str, Any] = {
    'collections': ["CMIP6_ensemble_median_TAS"],
    'datetime': "1951-01-01T00:00:00Z",
    'bbox': [-180, -90, 180, 90]
}

xyz_tile = (0,0,0)

## Profile titiler-xarray

In [7]:
@profile(add_to_return=True, quiet=False, log_library='s3fs')
def tile(src_path: str, x: int, y: int, z: int, *, variable: str, time_slice: str = None, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        z=z,
        decode_times=False,
        **kwargs,
    ) as dataset:
        dataarray, _ = update_dataset(dataset, variable=variable, time_slice=time_slice)
        
        with XarrayReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)

In [8]:
image_and_assets, logs = tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    *xyz_tile,
    #time_slice = search_dict['datetime'],
    reference=False,
    variable="TS",
)
logs

Setting up s3fs instance
RC: caching enabled? False (explicit option is False)
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/.zmetadata'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lat/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lon/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/1'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/10'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/11'}
CALL: get_object - () - {'Bucket': 'power-

KeyboardInterrupt: 

In [ ]:
import fsspec
import xarray as xr
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=False)
ds=xr.open_dataset(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    engine='zarr', consolidated=True)
ds

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im

## Profile titiler-pgstac

In [10]:
import pgstac.profile_pgstac as profile_pgstac 

Opening self.input:  s3://climatedashboard-data/cmip6/monthly/CMIP6_ensemble_median/tas/tas_month_ensemble-median_historical_195004.tif


AttributeError: 'Timer' object has no attribute 'from_start'

In [ ]:
image_and_assets, logs = tile(*xyz_tile)
logs

In [ ]:
content = image_and_assets[0].render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im